In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap, ListedColormap
import seaborn as sns

from benchmark_functions import AbstractCont

from olympus.surfaces import Surface

In [3]:
# Golem colormap
_reference_colors = ['#008080', '#70a494', '#b4c8a8', '#f6edbd', '#edbb8a', '#de8a5a','#ca562c']
_cmap = LinearSegmentedColormap.from_list('golem', _reference_colors)
_cmap_r = LinearSegmentedColormap.from_list('golem_r', _reference_colors[::-1])
# plt.register_cmap(cmap=_cmap)
# plt.register_cmap(cmap=_cmap_r)

def get_golem_colors(n):
    _cmap = plt.get_cmap('golem')
    return [_cmap(x) for x in np.linspace(0, 1, n)]

def plot_contour(ax, X0, X1, y, xlims, ylims, vlims=[None, None], alpha=0.5, contour_lines=True, contour_labels=True, 
                 labels_fs=8, labels_fmt='%d', n_contour_lines=8, contour_color='k', contour_alpha=1, cbar=False, cmap='golem'):
    # background surface
    if contour_lines is True:
        contours = ax.contour(X0, X1, y, n_contour_lines, colors=contour_color, alpha=contour_alpha, linestyles='dashed')
        if contour_labels is True:
            _ = ax.clabel(contours, inline=True, fontsize=labels_fs, fmt=labels_fmt)
    mappable = ax.imshow(y, extent=[xlims[0],xlims[1],ylims[0],ylims[1]], 
                         origin='lower', cmap=cmap, alpha=alpha, vmin=vlims[0], vmax=vlims[1])
    
    if cbar is True:
        cbar = plt.colorbar(mappable=mappable, ax=ax, shrink=0.5)
        
    return mappable

def plot_constr_surface(surface, ax=None, N=100):
    
    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5))

    x0 = np.linspace(0,1,N)
    x1 = np.linspace(0,1,N)
    X0, X1 = np.meshgrid(x0,x1)
    X = np.array([X0.flatten(), X1.flatten()]).T
    y = np.array(surface.run(X)).flatten()
    Y = np.reshape(y, newshape=np.shape(X0))

    _ = plot_contour(ax, X0, X1, Y, xlims=[0,1], ylims=[0,1], alpha=1, contour_lines=True, contour_labels=True, 
                 labels_fs=8, labels_fmt='%d', n_contour_lines=8, contour_alpha=0.8, cbar=False, cmap='golem')
    for param in surface.minima:
        x_min = param['params']
        ax.scatter(x_min[0], x_min[1], s=200, marker='*', color='#ffc6ff', zorder=20)
    
    y_feas = np.array(surface.eval_constr(X))
    Y_feas = np.reshape(y_feas, newshape=np.shape(X0))
    ax.imshow(Y_feas, extent=[0,1,0,1], origin='lower', cmap='gray', alpha=0.5, interpolation='none')
    
def plot_constr_surface_cat(surface, ax=None, mark_min=False):
    
    if ax is None:
        fig, ax = plt.subplots(nrows=1, ncols=1, figsize=(5,5))
        
        
    domain = np.arange(surface.num_opts)
    Z = np.zeros((surface.num_opts, surface.num_opts))
    Z_feas = np.empty((surface.num_opts, surface.num_opts))
    X = []
    for x_index, x in enumerate(domain):
        for y_index, y in enumerate(domain):
            x = ['x_{}'.format(x_index), 'x_{}'.format(y_index)]
            X.append(x)
            loss_value = surface.run(x)
            feas_bool = surface.is_feasible(x)
    
            Z[y_index, x_index] = loss_value
            Z_feas[y_index, x_index] = feas_bool
            
    # scale Camel for visualization 
    print(type(surface.surface))
    if type(surface.surface).__name__ == 'CatCamel':
        Z = np.log(Z)

    ax.imshow(Z, origin='lower', cmap = plt.get_cmap('golem'))
    ax.imshow(Z_feas, origin='lower', cmap='gray', alpha=0.5, interpolation='none')
    
    if mark_min is True:
        X_best = surface.str2array(surface.best)
        _ = ax.scatter(X_best[0], X_best[1], marker='*', s=200, color='#ffc6ff', linewidth=2, zorder=20)

In [4]:
def sort_feas_frac(feas_frac):
    if 0.1 <= feas_frac < 0.3:
        bin_ = '0.1-0.3'
    elif 0.3 <= feas_frac < 0.5:
        bin_ = '0.3-0.5'
    elif 0.5 <= feas_frac < 0.7:
        bin_ = '0.5-0.7'
    elif 0.7 <= feas_frac < 0.9:
        bin_ = '0.7-0.9'
    else:
        bin_ = None
        
    return bin_

In [ ]:
# bin surfaces circles

surfs_to_use = [
        'Dejong', 'StyblinskiTang', 'Branin', 'HyperEllipsoid', 'Schwefel',
        'Michalewicz', 'Rastrigin',
    ]

bins = {'0.1-0.3': {
            'objs': [], 'vals':[],
            },
        '0.3-0.5': {
            'objs': [], 'vals':[],
            },
        '0.5-0.7': {
            'objs': [], 'vals':[],
            },
        '0.7-0.9': {
            'objs': [], 'vals':[],
            },
}

num_per_bin = 40

bin_pops = [0,0,0,0]
seed_counter = 0
while not bin_pops ==[num_per_bin for _ in range(len(bins.keys()))]:
    
    surf_kind = np.random.choice(surfs_to_use)
    
    c = AbstractCont(
        surface_type=str(surf_kind),
        n_dims=2,
        constraint_type='circles',
        num_constraints=np.random.randint(4,12),
        constraint_params=[
            np.random.uniform(0.05,0.15),
            np.random.uniform(0.15, 0.4)
        ],
        seed=seed_counter,
    )
    feas_frac = c.estimate_feas_frac()
    bin_ = sort_feas_frac(feas_frac)
    if bin_:
        if len(bins[bin_]['objs']) < num_per_bin:
            # add the surface
            bins[bin_]['objs'].append(c)
            bins[bin_]['vals'].append(feas_frac)
        else:
            pass
    else:
        pass
    
    bin_pops = [len(bins[k]['objs']) for k in bins.keys()]
    print('bin_pops :', bin_pops)
    seed_counter+=1
    
    
    
    

In [ ]:
# save the surfaces to disk
pickle.dump(bins, open('feas_ablation/circles/surfaces.pkl', 'wb'))

In [5]:
# squares


surfs_to_use = [
        'CatDejong', 'CatCamel', 'CatMichalewicz', 'CatAckley',
    ]

bins = {'0.1-0.3': {
            'objs': [], 'vals':[],
            },
        '0.3-0.5': {
            'objs': [], 'vals':[],
            },
        '0.5-0.7': {
            'objs': [], 'vals':[],
            },
        '0.7-0.9': {
            'objs': [], 'vals':[],
            },
}

num_per_bin = 40

bin_pops = [0,0,0,0]
seed_counter = 0
while not bin_pops ==[num_per_bin for _ in range(len(bins.keys()))]:
    
    surf_kind = np.random.choice(surfs_to_use)
    
    c = AbstractCont(
        surface_type=str(surf_kind),
        n_dims=2,
        constraint_type='squares',
        num_constraints=np.random.randint(1,1000),
        constraint_params=[],
        seed=seed_counter,
    )
    feas_frac = c.estimate_feas_frac()
    bin_ = sort_feas_frac(feas_frac)
    if bin_:
        if len(bins[bin_]['objs']) < num_per_bin:
            # add the surface
            bins[bin_]['objs'].append(c)
            bins[bin_]['vals'].append(feas_frac)
        else:
            pass
    else:
        pass
    
    bin_pops = [len(bins[k]['objs']) for k in bins.keys()]
    print('bin_pops :', bin_pops)
    seed_counter+=1
    


[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`
[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`


bin_pops : [0, 0, 1, 0]
bin_pops : [1, 0, 1, 0]
bin_pops : [1, 1, 1, 0]
bin_pops : [1, 1, 1, 0]


[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`
[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`
[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`


bin_pops : [1, 2, 1, 0]


[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`


...

[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`


bin_pops : [40, 40, 40, 38]
bin_pops : [40, 40, 40, 39]
bin_pops : [40, 40, 40, 39]


[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`
[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`
[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`


bin_pops : [40, 40, 40, 39]
bin_pops : [40, 40, 40, 40]


[WARNING] Categorical surface chosen, but `num_opts` not defined. Setting `num_opts=21`


In [6]:
# save the surfaces to disk
pickle.dump(bins, open('feas_ablation/squares/surfaces.pkl', 'wb'))

In [ ]:
c = AbstractCont(
    surface_type='Dejong',
    n_dims=2,
    constraint_type='circles',
    num_constraints=5,
    constraint_params=[0.05, 0.15],
    seed=100701,
)

In [ ]:
plot_constr_surface(c)

In [ ]:
c.estimate_feas_frac()

In [ ]:
c = AbstractCont(
    surface_type='Schwefel',
    n_dims=2,
    constraint_type='rectangles',
    num_constraints=10,
    constraint_params=[0.05, 0.1],
    seed=100704,
)

In [ ]:
plot_constr_surface(c)

In [ ]:
c.estimate_feas_frac()

In [ ]:
surf = Surface(kind='CatDejong', num_opts=21)

In [ ]:
surf.num_opts

In [ ]:
c = AbstractCont(
    surface_type='CatCamel',
    n_dims=2,
    constraint_type='squares',
    num_constraints=20,
    constraint_params=[0.05, 0.1],
    seed=100704,
)

In [ ]:
c.estimate_feas_frac()

In [ ]:
plot_constr_surface_cat(c)